In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, LSTMNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch, train_super_batch_KL, train_super_batch_L1, train_regular_L1,train_regular_one_hot
from model_saver import model_saver, model_saver_wandb

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
def compare_model_to_bf(model:GRUNet,raceDB:Races,example_ct):
    with torch.no_grad():
        sft_max = nn.Softmax(dim=-1)
        l_sftmax = nn.LogSoftmax(dim=-1)
        nnl_loss = nn.KLDivLoss(reduction='batchmean')
        full_test_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))
        full_test_races_w_prices = []
        excluded, included = 0,0
        for r in full_test_races:
            if 0 in r.prices or -1 in r.prices:
                excluded+=1
            else:
                full_test_races_w_prices.append(r)
                included+=1
        print(included,excluded)

        output = l_sftmax(model(full_test_races_w_prices))
        bf_prices = torch.log(torch.tensor([x.implied_prob for x in full_test_races_w_prices ]).to('cuda:0'))
        full_classes = torch.stack([x.classes for x in full_test_races_w_prices ])

        print()

        print(f"our loss = {nnl_loss(output,full_classes)}")
        print(f"their loss = {nnl_loss(bf_prices ,full_classes)}")
        wandb.log({"our loss":nnl_loss(output,full_classes), "their loss":nnl_loss(bf_prices ,full_classes)})

In [7]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
hidden_size = 64
raceDB = build_dataset('new_windows_gru_REAL.npy', hidden_size ,state_filter="NSW")
raceDB.reset_all_lstm_states()

(1143056, 17)
(1143056, 17)
(282085, 20)
Latest date = 2023-02-08 00:00:00
num_features_per_dog=43


  0%|          | 0/14422 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/39580 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 39580, number of unique dogs = 14422


In [8]:
date = datetime.datetime.strptime("2022-10-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 34791, Test examples 4789


In [9]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [10]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack - AUS Testing", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = LSTMNet(input_size,config['hidden_size'])
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      criterion = nn.CrossEntropyLoss(reduction='none')
      #criterion = nn.SmoothL1Loss(reduction='none', beta=0.1)
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',threshold=0.0001, patience=50, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train_regular(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict, model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [11]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':25}

In [12]:
# raceDB.detach_all_hidden_states()
# raceDB.create_test_split()
# raceDB.reset_all_lstm_states()

In [13]:
# saved_dict = torch.load(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\long nsw 2000 RUN\long nsw 2000 RUN_450.pt")
# model_state_dict = saved_dict['model_state_dict']
# raceDB.fill_hidden_states_from_dict(saved_dict['db'])
# raceDB.to_cuda()
# # # predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # # predDB.to_cuda()

In [14]:
raceDB.detach_all_hidden_states()

In [15]:
1/float(10**-3)

1000.0

In [16]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [50]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [100,500,1000,5000]
        },
        "batch_before_backwards": {
            'values': [3,5,10,20,25]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [3, 5, 10, 20, 25]},
                'batch_size': {'values': [100, 500, 1000, 5000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [50]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 1e-05},
                'len_data': {'value': 39580},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 39580},
  'hidden_size': {'value': 64},
  'epochs': {'values': [50]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [100, 500, 1000, 5000]},
  'batch_before_backwards': {'values': [3, 5, 10, 20, 25]}}}

In [17]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: kk6pyc8x
Sweep URL: https://wandb.ai/nickojelly/LSTM_sweeps/sweeps/kk6pyc8x


wandb: Agent Starting Run: mdma0xjl with config:
wandb: 	batch_before_backwards: 10
wandb: 	batch_size: 100
wandb: 	dropout: 0.3
wandb: 	epochs: 50
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0003397173037515624
wandb: 	len_data: 39580
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 10, 'batch_size': 100, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0003397173037515624, 'len_data': 39580, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
50
{'batch_before_backwards': 10, 'batch_size': 100, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0003397173037515624, 'len_data': 39580, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
LSTMNet(
  (batchnorm): BatchNorm1d(416, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=416, out_features=416, bias=True)
  (lstm1): LSTMCell(416, 64)
  (lstm2): LSTMCell(416, 64)
  (lstm3): LSTMCell(416, 64)
  (lstm4): LSTMCell(416, 64)
  (lstm5): LSTMCell(416, 64)
  (lstm6): LSTMCell(416, 64)
  (lstm7): LSTMCell(416, 64)
  (lstm8): LSTMCell(416, 64)
  (rl1

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/347 [00:00<?, ?it/s]

  0%|          | 0/478 [00:00<?, ?it/s]

  0%|          | 0/347 [00:00<?, ?it/s]

  0%|          | 0/478 [00:00<?, ?it/s]

  0%|          | 0/347 [00:00<?, ?it/s]

In [ ]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
stop
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

NameError: name 'stop' is not defined

In [ ]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

In [ ]:

dataset.create_hidden_states_dict()